In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch tutorial from https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [139]:
training_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [140]:
batch_size = 64

# Why does the batch size matter? What is the importance of 64?
# DataLoader "wraps an iterable" over the dataset, presumably for use in a loop, map, etc.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [141]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [142]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define our model
# Inherits from nn.Module (so nn definitively stands for "neural network")
# We're creating the layers and structure of the neural network here.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # why 512? I get why 28*28 is the feature input.
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10) # 10 classes.
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits # is the name "logits" significant here? yes: https://deepai.org/machine-learning-glossary-and-terms/logit

model = NeuralNetwork().to(device)

Using cpu device


In [143]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # lr = "learning rate"

In [150]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # primes the model for training. Can't tell what this does in the source other than set a flag to True
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute the prediction error of our model
        pred = model(X)
        loss = loss_fn(pred, y)

        # Propagate the loss back through the model's parameters with the optimizer
        optimizer.zero_grad() # zero the gradients - not sure why we need to do this
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # set the model to "evaluation" mode. presumably this is different to "training" mode.
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [152]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("DONE!")

Epoch 1
--------------------
loss: 1.043253  [    0/60000]
loss: 1.067827  [ 6400/60000]
loss: 0.873667  [12800/60000]
loss: 1.027742  [19200/60000]
loss: 0.900610  [25600/60000]
loss: 0.924967  [32000/60000]
loss: 0.976301  [38400/60000]
loss: 0.922535  [44800/60000]
loss: 0.953156  [51200/60000]
loss: 0.901692  [57600/60000]
Test Error: 
 Accuracy: 66.9%, Avg loss: 0.910591 

Epoch 2
--------------------
loss: 0.957439  [    0/60000]
loss: 1.000268  [ 6400/60000]
loss: 0.791617  [12800/60000]
loss: 0.962625  [19200/60000]
loss: 0.840908  [25600/60000]
loss: 0.856907  [32000/60000]
loss: 0.923343  [38400/60000]
loss: 0.873875  [44800/60000]
loss: 0.896018  [51200/60000]
loss: 0.854331  [57600/60000]
Test Error: 
 Accuracy: 68.2%, Avg loss: 0.858261 

Epoch 3
--------------------
loss: 0.890890  [    0/60000]
loss: 0.949567  [ 6400/60000]
loss: 0.729614  [12800/60000]
loss: 0.914470  [19200/60000]
loss: 0.797841  [25600/60000]
loss: 0.805576  [32000/60000]
loss: 0.883109  [38400/60000]

In [153]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model.train()
for param in model.parameters():
    print(type(param), param.size())

for X, y in train_dataloader:
    # X - the input data
    # Y - the 
    X.to(device), y.to(device)
    pred = model(X)
    print(y.size())
    print(pred.size())
    print(pred)
    print(y)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(loss)
    break

In [91]:
x = torch.tensor([1, 2, 3])
torch.sum(x)

#input = torch.randn(4, 4)
input = torch.tensor([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
])
input



tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [101]:
random_seed = 1
torch.manual_seed(random_seed) # without this, nn.Linear will produce different results each time.
m = nn.Linear(3, 4, False)
output = m(input)
output

tensor([[ 0.2975,  0.2710, -0.1188, -0.0707],
        [-0.2548, -0.5435,  0.2937,  0.1601],
        [-0.1119,  0.3462,  0.0803,  0.0285]], grad_fn=<MmBackward0>)